## [ Chapter 13 - Semantic Search with Dense Vectors ] 
## Setting up the outdoors dataset

We're going to use the Outdoors dataset for this chapter, and for a very important reason: the vocabulary and contexts in the outdoor question and answer __domain__ already have good coverage in the Transformer models we'll be using.

This is because the datasets that were used to train the model include sources that are likely to have similar subject matter.  Wikipedia was used to train bert-base-uncased (https://huggingface.co/bert-base-uncased#training-data) and, surprise! wikipedia has a section specifically on outdoors content: https://en.wikipedia.org/wiki/Outdoor

This is important, because if the words and their contexts haven't been seen before, the model will be less accurate.

Also, who doesn't enjoy playing around with a new dataset?! Data is search nerd candy.

In [ ]:
import sys
import pickle
import json
sys.path.append('..')
from aips import *
from IPython.display import display,HTML
engine = get_engine()

## Listing 13.1

### Creating our Collection and Indexing the documents

In [ ]:
#outdoors
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../../data/outdoors/' && tar -xvf outdoors.tgz -C '../../data/outdoors/'

Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses.csv
._roberta-base-squad2-outdoors
roberta-base-squad2-outdoors/
roberta-base-squad2-outdoors/._tokenizer_config.json
roberta-base-squad2-outdoors/tokenizer_config.json
roberta-base-squad2-outdoors/._special_tokens_map.json
roberta-base-squad2-outdoors/special_tokens_map.json
roberta-base-squad2-outdoors/._config.json
roberta-base-

## Listing 13.2

### Querying the outdoors dataset with a noun phrase

In [ ]:
from densevectors.outdoors import *
#Transform the the outdoors.csv file into solr documents
outdoors_collection = engine.create_collection("outdoors")
outdoors_dataframe = load_outdoors_data("../data/outdoors/posts.csv")
outdoors_collection.write(outdoors_dataframe)

outdoors_dataframe.show(5)

Wiping "outdoors" collection
Creating "outdoors" collection
Status: Success


/home/jovyan/notebooks/densevectors/outdoors.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(h, 'lxml')
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+------------+------------------+---------+--------------------+-------------+-----+----------+--------------------+-------------+------------------+-------------------+------------------------+--------------------+--------------------+--------------------+--------------------+------------+-------------+--------------+-----------+--------------------+--------------------+
| id|post_type_id|accepted_answer_id|parent_id|       creation_date|deletion_date|score|view_count|                body|owner_user_id|owner_display_name|last_editor_user_id|last_editor_display_name|      last_edit_date|  last_activity_date|               title|                tags|answer_count|comment_count|favorite_count|closed_date|community_owned_date|                 url|
+---+------------+------------------+---------+--------------------+-------------+-----+----------+--------------------+-------------+------------------+-------------------+------------------------+--------------------+--------------------+--

In [ ]:
def query_collection(query):
    request = {"query": query,
               "query_fields": ["title", "body"],
               "return_fields": ["id", "title", "url"],
               "filters": [("post_type_id", "1")],
               "hightlight": True}
    return outdoors_collection.search(**request)
    
def display_html(response, verbose=False):    
    highlights = [response["highlighting"][k] for k in response["highlighting"].keys()]
    for r in zip(response["docs"], highlights):
        print(r)
        display(HTML('<a href="'+r[0]["url"]+'">qid '+r[0]["id"]+'</a>&nbsp;'))
        if "title" in r[0].keys():
            display(HTML("<strong>Question Title:</strong>" + r[0]["title"]))
        if verbose:
            display(HTML("<strong>Body:</strong>" + html.unescape(str(r[1]["body"][0]))))
            display(HTML("<hr>"))

## Listing 13.2

### Querying our collection with a noun phrase

In [ ]:
response = query_collection("climbing knots")
display_html(response, verbose=True)

In [ ]:
response = query_collection("snow camping safety")
display_html(response, verbose=True)

## Listing 13.3

### Querying our collection with a question

In [ ]:
response = query_collection("What is DEET?")
display_html(response, verbose=True)

Up next: [Introduction to Transformers](2.introduction-to-transformers.ipynb)